In [4]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import random
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
from syn_DI_dataset import make_dataset, make_dataloader

In [5]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
with open('config.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

['rgb', 'depth', 'lidar', 'mmwave']
S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16'

In [6]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    all_actions = {'A01': 0., 'A02': 1., 'A03': 2., 'A04': 3., 'A05': 4., 
                'A06': 5., 'A07': 6., 'A08': 7., 'A09': 8., 'A10': 9.,
                'A11': 10., 'A12': 11., 'A13': 12., 'A14': 13., 'A15': 14., 
                'A16': 15., 'A17': 16., 'A18': 17., 'A19': 18., 'A20': 19., 
                'A21': 20., 'A22': 21., 'A23': 22., 'A24': 23., 'A25': 24., 
                'A26': 25., 'A27': 26.}
    # kpts = []
    # [kpts.append(np.array(t['output'])) for t in batch]
    # kpts = torch.FloatTensor(np.array(kpts))
    labels = []
    [labels.append(all_actions[t['action']]) for t in batch]
    labels = torch.FloatTensor(labels)

    # lengths = torch.tensor([t['input_mmwave'].shape[0] for t in batch ])

    # rgb
    rgb_data = np.array([(t['input_rgb']) for t in batch ])
    rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # depth
    depth_data = np.array([(t['input_depth']) for t in batch ])
    depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # mmwave
    ## padd
    mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    ## compute mask
    mmwave_data = mmwave_data.permute(1,0,2)

    # lidar
    ## padd
    lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    ## compute mask
    lidar_data = lidar_data.permute(1,0,2)

    # # wifi-csi
    # wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    # wifi_data = torch.FloatTensor(wifi_data)
    "要改"
    exist_list = [True, False, False, False]

    return rgb_data, depth_data, lidar_data, mmwave_data, labels, exist_list
    # return rgb_data, kpts, exist_list

In [8]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['train_loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['val_loader'], collate_fn = collate_fn_padd)

In [9]:
from baseline.baseline_model import single_model

# model = model(['RGB'])
# "modality_list: list of string, choose from ['RGB', 'Depth', 'mmWave', 'Lidar', 'Wifi']"
rgb_model = single_model(['rgb'])
# rgb_weights = torch.load('./baseline/baseline_weights/Single/RGB.pt')
# print(rgb_weights.keys())
# print(rgb_model)
# print(rgb_model.state_dict().keys())
rgb_model.load_state_dict(torch.load('./baseline/HAR_baseline_weights/Single/rgb_.pt'))
rgb_model.cuda()

depth_model = single_model(['depth'])
depth_model.load_state_dict(torch.load('./baseline/HAR_baseline_weights/Single/depth_.pt'))
depth_model.cuda()

mmwave_model = single_model(['mmwave'])
mmwave_model.load_state_dict(torch.load('./baseline/HAR_baseline_weights/Single/mmwave_.pt'))
mmwave_model.cuda()

lidar_model = single_model(['lidar'])
lidar_model.load_state_dict(torch.load('./baseline/HAR_baseline_weights/Single/lidar_.pt'))
lidar_model.cuda()

single_model(
  (feature_extractor): single_feature_extrator(
    (lidar_extractor): lidar_feature_extractor(
      (fc1): Sequential(
        (0): Linear(in_features=3, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=32, bias=True)
      )
      (transformer1): TransformerBlock(
        (fc1): Linear(in_features=32, out_features=512, bias=True)
        (fc2): Linear(in_features=512, out_features=32, bias=True)
        (fc_delta): Sequential(
          (0): Linear(in_features=3, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=512, bias=True)
        )
        (fc_gamma): Sequential(
          (0): Linear(in_features=512, out_features=512, bias=True)
          (1): ReLU()
          (2): Linear(in_features=512, out_features=512, bias=True)
        )
        (w_qs): Linear(in_features=512, out_features=512, bias=False)
        (w_ks): Linear(in_features=512, out_features=512, bias=False

In [19]:
def get_result(rgb_model,depth_model,mmwave_model,lidar_model, tensor_loader, device):
    rgb_model.eval()
    depth_model.eval()
    mmwave_model.eval()
    lidar_model.eval()
    # test_mpjpe = 0
    # test_pampjpe = 0
    # test_mse = 0
    # random.seed(config['modality_existances']['val_random_seed'])
    for i, data in tqdm(enumerate(tensor_loader)):
        # start_time = time.time()
        rgb_data, depth_data, lidar_data, mmwave_data, label, exist_list = data
        rgb_data = rgb_data.to(device)
        depth_data = depth_data.to(device)
        lidar_data = lidar_data.to(device)
        mmwave_data = mmwave_data.to(device)
        label.to(device)
        labels = label.type(torch.FloatTensor)
        # outputs = model(input_1, exist_list)
        # outputs = outputs.type(torch.FloatTensor)
        rgb_model.to('cuda')
        rgb_outputs = rgb_model(rgb_data, [True, False, False, False, False])
        rgb_model.to('cpu')
        del rgb_data
        depth_model.to('cuda')
        depth_outputs = depth_model(depth_data, [False, True, False, False, False])
        depth_model.to('cpu')
        del depth_data
        mmwave_model.to('cuda')
        mmwave_outputs = mmwave_model(mmwave_data, [False, False, True, False, False])
        mmwave_model.to('cpu')
        del mmwave_data
        lidar_model.to('cuda')
        lidar_outputs = lidar_model(lidar_data, [False, False, False, True, False])
        lidar_model.to('cpu')
        del lidar_data
        
        # rgb_outputs = rgb_outputs.to(device)
        rgb_outputs = rgb_outputs.detach().cpu().numpy()
        depth_outputs = depth_outputs.detach().cpu().numpy()
        mmwave_outputs = mmwave_outputs.detach().cpu().numpy()
        lidar_outputs = lidar_outputs.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        
        if i == 0:
            rgb_result = rgb_outputs
            depth_result = depth_outputs
            mmwave_result = mmwave_outputs
            lidar_result = lidar_outputs
            all_label = labels
            print(rgb_outputs.shape, depth_outputs.shape, mmwave_outputs.shape, lidar_outputs.shape, labels.shape)
        else:
            rgb_result = np.vstack((rgb_result, rgb_outputs))
            depth_result = np.vstack((depth_result, depth_outputs))
            mmwave_result = np.vstack((mmwave_result, mmwave_outputs))
            lidar_result = np.vstack((lidar_result, lidar_outputs))
            all_label = np.hstack((all_label, labels))
            if i ==1:
                print(rgb_result.shape, depth_result.shape, mmwave_result.shape, lidar_result.shape, all_label.shape)

    np.save('./baseline_results/rgb_result.npy', rgb_result)
    np.save('./baseline_results/depth_result.npy', depth_result)
    np.save('./baseline_results/mmwave_result.npy', mmwave_result)
    np.save('./baseline_results/lidar_result.npy', lidar_result)
    np.save('./baseline_results/all_label.npy', all_label)

    return

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
get_result(rgb_model,depth_model,mmwave_model,lidar_model, val_loader, device)

1it [00:01,  1.29s/it]

(8, 27) (8, 27) (8, 27) (8, 27) (8,)


2it [00:01,  1.09it/s]

(16, 27) (16, 27) (16, 27) (16, 27) (16,)


6805it [1:06:12,  1.71it/s]


In [21]:
rgb_result=np.load('./baseline_results/rgb_result.npy')
depth_result=np.load('./baseline_results/depth_result.npy')
mmwave_result=np.load('./baseline_results/mmwave_result.npy')
lidar_result=np.load('./baseline_results/lidar_result.npy')
# wifi_result=np.load('./baseline_results/wifi_result.npy')
all_label=np.load('./baseline_results/all_label.npy')

In [29]:
predict_y = np.argmax(rgb_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('rgb accuracy:', epoch_accuracy)

rgb accuracy: 0.25265923245090294


In [30]:
predict_y = np.argmax(depth_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('depth accuracy:', epoch_accuracy)

depth accuracy: 0.4992192236327228


In [31]:
predict_y = np.argmax(mmwave_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('mmwave accuracy:', epoch_accuracy)

mmwave accuracy: 0.8483089302445208


In [32]:
predict_y = np.argmax(lidar_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('lidar accuracy:', epoch_accuracy)

lidar accuracy: 0.6383260154685577


In [33]:
ID_result = (rgb_result+depth_result)/2
predict_y = np.argmax(ID_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('ID accuracy:', epoch_accuracy)

ID accuracy: 0.4762184704131685


In [34]:
IL_result = (rgb_result+lidar_result)/2
predict_y = np.argmax(IL_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('IL accuracy:', epoch_accuracy)

IL accuracy: 0.47537339481564495


In [35]:
IR_result = (rgb_result+mmwave_result)/2
predict_y = np.argmax(IR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('IR accuracy:', epoch_accuracy)

IR accuracy: 0.7642055370822847


In [36]:
DL_result = (depth_result+lidar_result)/2
predict_y = np.argmax(DL_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('DL accuracy:', epoch_accuracy)

DL accuracy: 0.6054415519997061


In [37]:
DR_result = (depth_result+mmwave_result)/2
predict_y = np.argmax(DR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('DR accuracy:', epoch_accuracy)

DR accuracy: 0.7990373486671688


In [39]:
LR_result = (lidar_result+mmwave_result)/2
predict_y = np.argmax(LR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('LR accuracy:', epoch_accuracy)

LR accuracy: 0.8978744511601419


In [40]:
IDL_result = (rgb_result+depth_result+lidar_result)/3
predict_y = np.argmax(IDL_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('IDL accuracy:', epoch_accuracy)

IDL accuracy: 0.5623059541087209


In [41]:
IDR_result = (rgb_result+depth_result+mmwave_result)/3
predict_y = np.argmax(IDR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('IDR accuracy:', epoch_accuracy)

IDR accuracy: 0.7159811143975162


In [42]:
ILR_result = (rgb_result+lidar_result+mmwave_result)/3
predict_y = np.argmax(ILR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('ILR accuracy:', epoch_accuracy)

ILR accuracy: 0.8233975713262176


In [44]:
DLR_result = (depth_result+lidar_result+mmwave_result)/3
predict_y = np.argmax(DLR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('DLR accuracy:', epoch_accuracy)

DLR accuracy: 0.8345488949718002


In [45]:
IDLR_result = (rgb_result+depth_result+lidar_result+mmwave_result)/4
predict_y = np.argmax(IDLR_result,axis=1)
epoch_accuracy = (predict_y == all_label).sum() / all_label.size
print('IDLR accuracy:', epoch_accuracy)

IDLR accuracy: 0.7650506126798082
